## ParentDocumentRetriever

### Creazione Vector-Store

In [2]:
# lettura di tutti i documenti da una cartella

from langchain_community.document_loaders import DirectoryLoader 

loader = DirectoryLoader('data/dialoghi', glob="**/*.txt")
documents = loader.load()

for doc in documents:
    print(doc, "\n\n")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


page_content='Al negozio di abbigliamento

Commesso – Buongiorno, posso aiutarla? Cliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli? Commesso – Certo, che taglia? Cliente – Porto una 50. Commesso – Eccoli qua!

Cliente – Grazie… però prima, posso dire che finalmente oggi si respira un po’? Ieri sembrava luglio! Commesso – Ah, guardi, non me ne parli. Stamattina ho già cambiato due volte la maglietta, da quanto fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una serra. Cliente – Già, e poi è arrivato quel temporale all’improvviso, proprio mentre tornavo a casa. Mi sono bagnato fino alle ossa. Commesso – Eh sì, classico. Qui appena arriva un po’ di caldo, dopo due giorni parte il temporale estivo. Però oggi sembra tenere, almeno per ora. Cliente – Speriamo, che devo ancora fare un giro in centro dopo.

Commesso – A proposito, è appassionato di sport? Ieri notte ho guardato la partita dei Lakers contro i Celtics, non riuscivo a dormire! Clie

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())

# ParentDocumentRetriever
### Recupero di un Chunk di un Documento e del Documento che lo contiene 

In [4]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers import ParentDocumentRetriever

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(collection_name="full_documents", embedding_function=OpenAIEmbeddings())

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

retriever.add_documents(documents, ids=None)

list(store.yield_keys())

['c714a21a-d4cc-420a-a456-689ee9bdf475',
 'a31757c6-bbaa-4041-b921-39e63192de49',
 '38835212-3785-4a35-8300-eef8aa3fb20b',
 '6e16b1d1-2cdb-470f-b417-5f0f44c94464',
 'c4e5b32f-bf86-4643-bb9d-89efce709e45']

In [5]:
sub_docs = vectorstore.similarity_search("in quale dialogo ci sono delle mamme che parlano?")

sub_docs[0]

Document(metadata={'doc_id': '38835212-3785-4a35-8300-eef8aa3fb20b', 'source': 'data\\dialoghi\\dialogo3.txt'}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.")

In [6]:
retrieved_docs = retriever.invoke("in quale dialogo ci sono delle mamme che parlano?")

retrieved_docs[0]

Document(metadata={'source': 'data\\dialoghi\\dialogo3.txt'}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.\n\nMamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.\n\nMamma 2 - Avete sentito della nuova maestra di inglese? Dicono che s

## Gestione di Chunk multi-livello

*Recuperare piccole parti di un chunk, per poi recuperare il chunk da cui sono state generate.*    
    
Quando si splittano i documenti per il RAG, siamo di fronte, spesso, a desideri contrastanti:
* Si vuole avere documenti di piccole dimensioni, in modo che i loro embeddings possano rispecchiare fedelmente il significato di ogni documento; se il testo/documento è troppo lungo, i vettori di embeddings potrebbero perdere significato
* Si vuole avere documenti abbastanza lunghi da contenere tutto il contesto utile alla generazione di una risposta efficace.
    
**ParentDocumentRetriever** raggiunge questo equilibrio suddividendo e archiviando chunk su più livelli di dettaglio. Durante il recupero, sfrutta i chunk più piccoli per trovare i documenti meglio attinenti alla ricerca effettuata, ma poi usa gli ID genitore di questi chunk per restituire i documenti più grandi.

In [7]:
# Text-Splitter per i chunk di documento più grandi, da usare come dati recuperati
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=150)

# Secondo Text-Splitter, per la creazione di chunk più piccoli su cui inidicizzare le ricerche
child_splitter = RecursiveCharacterTextSplitter(chunk_size=85, chunk_overlap=15)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())

# dizionario in memoria utilizzato per recuperare i documenti più grandi, una volta
# identificati tramite ricerca sui chunk più piccoli
store = InMemoryStore()

In [9]:
# ParentDocumentRetriever gestisce automaticamente
# il "doppio livello" di granularità dei chunk

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(documents)

In [10]:
# dai 5 documenti di partenza, vengono creati 
# 64 chunk da 350 caratteri massimo
# memorizzati in `store`, mentre i chunk più
# piccoli, da massimo 85 caratteri, sono
# memorizzati nel vector-store
len(list(store.yield_keys()))

128

In [11]:
store.store

{'19fbfbb1-e8a3-4542-8e20-dc501b774b22': Document(metadata={'source': 'data\\dialoghi\\dialogo.txt'}, page_content='Al negozio di abbigliamento\n\nCommesso – Buongiorno, posso aiutarla? Cliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli? Commesso – Certo, che taglia? Cliente – Porto una 50. Commesso – Eccoli qua!'),
 '25db8757-7d26-48af-a9f8-e7c45bd720f5': Document(metadata={'source': 'data\\dialoghi\\dialogo.txt'}, page_content='Cliente – Grazie… però prima, posso dire che finalmente oggi si respira un po’? Ieri sembrava luglio! Commesso – Ah, guardi, non me ne parli. Stamattina ho già cambiato due volte la maglietta, da quanto fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una serra. Cliente – Già, e poi è arrivato quel temporale all’improvviso, proprio'),
 '6ce81e7b-1919-48d8-85bc-4c65cebd6836': Document(metadata={'source': 'data\\dialoghi\\dialogo.txt'}, page_content='fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una s

In [12]:
# accedendo al VectrorStore (Chroma), tramite similarity search, la ricerca viene effettuata
# sui chunk estratti più piccoli
sub_docs = vectorstore.similarity_search("in quale dialogo ci sono delle mamme che parlano?")

sub_docs[0]

Document(metadata={'doc_id': 'fea74f9e-8dd1-4c74-ac58-5233543f9111', 'source': 'data\\dialoghi\\dialogo3.txt'}, page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 -')

In [13]:
for s in sub_docs:
    print(s)

page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 -' metadata={'doc_id': 'fea74f9e-8dd1-4c74-ac58-5233543f9111', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 -' metadata={'doc_id': '28f1f15e-50c7-41e1-917d-52ba7d4ad709', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1' metadata={'doc_id': '1264c18c-e052-4466-9453-dbe4137de7ec', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1' metadata={'doc_id': '3dab3cc3-8957-4ae0-8333-d579326a7dea', 'source': 'data\\dialoghi\\dialogo3.txt'}


In [14]:
# il retriever ritorna invece il chunk di documento più grande
retrieved_docs = retriever.invoke("in quale dialogo ci sono delle mamme che parlano?")

retrieved_docs[0]

Document(metadata={'source': 'data\\dialoghi\\dialogo3.txt'}, page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 - Dovrebbe essere la prossima settimana, credo. Devo ancora controllare il calendario degli eventi. Mamma 4 - Sì, è giovedì prossimo alle 17:00. Hanno detto che parleranno dei progressi dei bambini e dei prossimi progetti scolastici. Mamma 1 - Perfetto, così posso')

In [15]:
for r in retrieved_docs:
    print(r)

page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 - Dovrebbe essere la prossima settimana, credo. Devo ancora controllare il calendario degli eventi. Mamma 4 - Sì, è giovedì prossimo alle 17:00. Hanno detto che parleranno dei progressi dei bambini e dei prossimi progetti scolastici. Mamma 1 - Perfetto, così posso' metadata={'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 - Dovrebbe essere la prossima settimana, credo. Devo ancora controllare il calendario degli eventi. Mamma 4 - Sì, è giovedì prossimo alle 17:00. Hanno detto che parleranno dei progressi dei bambini e dei prossimi progetti scolastici. Mamma 1 - Perfetto, così posso' metadata={'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='Mamma 2 - Cambiando argomento, cosa farete questo fine settimana? Mamma 3 - Noi andiamo al parco, sperando che il tempo sia bello. I bambini hanno bisogno